## Temperature Ramp (5.0 -> 5.2)

In [1]:
#-----Import some essential packages

from __future__ import division
import hoomd
import hoomd.md

#-----Some important variables

t_0 = 5.0;
t_1 = 5.2;
p_max  = 1.4;
steps_ramp = 1e5;

#-----Crea un contexto de simulación

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Extrae la configuración del centro de masa de las partículas

system = hoomd.init.read_gsd("T_CM_5.0_P_1.4_equilibrium.gsd",frame = -1)

#-----Define cada mesógeno en el marco de referencia local de cada centro de masa.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Declara que las moléculas creadas son cuerpos rígidos

rigid.create_bodies();

#-----Define la energía potencial

nl = hoomd.md.nlist.tree();

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist = nl)
lj.set_params(mode='shift')

#-----Define la interacción entre las especies de la simulación

lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0)

#-----Selecciona un integrador standard

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define el grupo sobre el cual se integraran las ecuaciones de movimiento

mesogens = hoomd.group.rigid_center();

HOOMD-blue 2.6.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/29/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 1000 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for mol

In [2]:
#----Integrate using NPT

temp = hoomd.variant.linear_interp(points = [(0,t_0), (steps_ramp, t_1)])

npt = hoomd.md.integrate.npt(group = mesogens, kT = temp, tau = 1.0, tauP = 9.5, P = p_max);
npt.randomize_velocities(seed = 42)

#-----Write output and Run the Simulation

log_file = "T_" + str(t_1) + "_P_" + str(p_max) + "_ramp.log"
gsd_file = "T_" + str(t_1) + "_P_" + str(p_max) + "_ramp.gsd" 
meso_gsd_file = "T_CM_" + str(t_1) + "_P_" + str(p_max) + "_ramp.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite = True); 

In [ ]:
hoomd.run(steps_ramp)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 955136 / 1055000 | TPS 13.5942 | ETA 02:02:26
Time 00:00:20 | Step 955264 / 1055000 | TPS 12.7529 | ETA 02:10:20
Time 00:00:30 | Step 955419 / 1055000 | TPS 15.4949 | ETA 01:47:06
Time 00:00:40 | Step 955593 / 1055000 | TPS 17.3788 | ETA 01:35:20
Time 00:00:50 | Step 955758 / 1055000 | TPS 16.4419 | ETA 01:40:35
Time 00:01:00 | Step 955899 / 1055000 | TPS 13.9915 | ETA 01:58:02
Time 00:01:10 | Step 956038 / 1055000 | TPS 13.8808 | ETA 01:58:49
Time 00:01:20 | Step 956203 / 1055000 | TPS 16.4774 | ETA 01:39:55
Time 00:01:30 | Step 956387 / 1055000 | TPS 18.3972 | ETA 01:29:20
Time 00:01:40 | Step 956572 / 1055000 | TPS 18.3602 | ETA 01:29:20
Time 00:01:50 | Step 956720 / 1055000 | TPS 14.7995 | ETA 01:50:40
Time 00:02:00 | Step

In [ ]:
system.get_metadata()